In [11]:
import os
import sys
sys.path.append( "../")
from src.utils import solve_pmedian_with_balance, add_p_via_mip, relocate_via_mip, relocate_via_greedy,greedy_add,preprocess, add_p_via_mip_multi
from src.models import SurvivalRegressionForecaster
import importlib
import joblib
import pandas as pd
import geopandas as gpd
import numpy as np
import joblib
from itertools import product
import requests

In [12]:
# OSRM base URL
OSRM_URL = "http://localhost:8080/route/v1/driving"

In [13]:
old_d=True

Todo

Data preprocessing
Multiple Firestations within a single grid


LOADING DATASET and Models

In [14]:
grids= gpd.read_file('../sample_data/grids.shp')
grids['x']= grids['geometry'].centroid.x
grids['y']= grids['geometry'].centroid.y
survival_data= pd.read_csv('../sample_data/survival_dataset.csv')
fire_stations= gpd.read_file('../sample_data/Fire_Station_Locations_-1484530934421591558.geojson')
fire_stations= gpd.sjoin(fire_stations, grids, how='inner', predicate='within')

/var/folders/tf/7k8vj9w17715wv9yqsqn7pm00000gq/T/ipykernel_9422/1104123106.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grids['x']= grids['geometry'].centroid.x
/var/folders/tf/7k8vj9w17715wv9yqsqn7pm00000gq/T/ipykernel_9422/1104123106.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grids['y']= grids['geometry'].centroid.y


In [15]:
model=joblib.load('../models/model.pkl')

DATA Preprocessing

In [16]:
cat=['window']
num=['historical_density']

train_df,test_df,features,scaler =preprocess(survival_data, cat, num)
train_df[features]=train_df[features].astype(float)
test_df[features]=test_df[features].astype(float)

In [17]:
test_df.reset_index(drop=True, inplace=True)

In [18]:
# half_count = len(test_df) // 4
# test_df.iloc[:half_count].to_csv('../sample_data/sample_incidents.csv', index=False)

In [19]:
# model.predict(train_df,{'features':features})
model.predict(test_df,{'features':features})

,Time,Latitude,Longitude,cell_id,weekday,historical_density,xd,weighted_population,weighted_housing_units,cluster_label,...,AlarmDate,PSAPDate,time_bet,window_1,window_2,window_3,window_4,window_5,original_historical_density,predicted_time_bet
0,2023-11-01 00:00:00,36.146819,-86.757259,323,0,0.483995,335,3668,1722,5,...,2023-11-01 00:04:23,2023-10-31 23:27:00.000,8.363611,0.0,0.0,0.0,0.0,0.0,0.028573,2.298654
1,2023-11-01 00:00:00,36.172264,-86.815118,250,0,-0.074961,40,3015,1283,7,...,2023-11-01 00:07:02,2023-11-01 00:06:46.000,19.011111,0.0,0.0,0.0,0.0,0.0,0.015812,1.148734
2,2023-11-01 00:00:00,36.135756,-86.787771,270,0,0.017206,122,6379,2523,7,...,2023-11-01 00:22:33,2023-11-01 00:22:12.000,1.593056,0.0,0.0,0.0,0.0,0.0,0.017916,0.968673
3,2023-11-01 00:00:00,36.158489,-86.777128,297,0,2.765679,416,2310,2244,2,...,2023-11-01 00:25:32,2023-11-01 00:25:13.000,0.825278,0.0,0.0,0.0,0.0,0.0,0.080664,0.811596
4,2023-11-01 00:00:00,36.192344,-86.729003,384,0,-0.524330,44,2862,1639,3,...,2023-11-01 00:40:31,2023-11-01 00:38:00.000,13.766667,0.0,0.0,0.0,0.0,0.0,0.005552,14.946698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18265,2023-12-31 23:00:00,36.160723,-86.778429,297,1,2.765679,416,2310,2244,2,...,2023-12-31 23:20:18,2023-12-31 23:20:18.000,0.025000,0.0,0.0,0.0,0.0,1.0,0.080664,0.558395
18266,2023-12-31 23:00:00,36.160950,-86.775805,297,1,2.765679,416,2310,2244,2,...,2023-12-31 23:21:48,2023-12-31 23:21:48.000,0.111111,0.0,0.0,0.0,0.0,1.0,0.080664,0.558395
18267,2023-12-31 23:00:00,36.161030,-86.777204,297,1,2.765679,416,2310,2244,2,...,2023-12-31 23:28:28,2023-12-31 23:28:28.000,0.185000,0.0,0.0,0.0,0.0,1.0,0.080664,0.558395
18268,2023-12-31 23:00:00,36.161589,-86.776560,297,1,2.765679,416,2310,2244,2,...,2023-12-31 23:39:34,2023-12-31 23:39:34.000,0.143611,0.0,0.0,0.0,0.0,1.0,0.080664,0.558395


In [20]:
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
OD_matrix=False
if OD_matrix:
    pairs = [(i,j) for i in test_df.index for j in grids.index]

    # Create merged DataFrame
    data = []
    for i_idx, f_idx in tqdm(pairs, total=len(pairs)):
        data.append({
            "incident_id": i_idx,
            "incident_lat": test_df.at[i_idx, 'Latitude'],
            "incident_lon": test_df.at[i_idx, 'Longitude'],
            "fire_station_id": f_idx,
            "fire_lat": grids.at[f_idx, 'y'],
            "fire_lon": grids.at[f_idx, 'x']
        })

    df_merged = pd.DataFrame(data)

    print("Merged DataFrame shape:", df_merged.shape)
    # Show a sample
    df_merged.head()


    # ————— Configuration —————

    MAX_WORKERS     = 10
    REQUEST_TIMEOUT = 5  # seconds

    # ————— Helper functions —————
    def build_osrm_url(o_lon, o_lat, d_lon, d_lat):
        return f"{OSRM_URL}/{o_lon},{o_lat};{d_lon},{d_lat}?overview=false"

    def get_travel_time_from_row(row: dict):
        """
        row must be a dict with keys
        'fire_lon','fire_lat','incident_lon','incident_lat'
        """
        url = build_osrm_url(
            row['fire_lon'], row['fire_lat'],
            row['incident_lon'], row['incident_lat']
        )
        try:
            resp = requests.get(url, timeout=REQUEST_TIMEOUT)
            resp.raise_for_status()
            return resp.json()['routes'][0]['duration']
        except Exception:
            return None

    # ————— Prepare inputs —————
    rows = df_merged[
        ['fire_lon','fire_lat','incident_lon','incident_lat']
    ].to_dict('records')
    travel_times = [None] * len(rows)

    # ————— Parallel fetch via map + tqdm —————
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        for i, duration in enumerate(
            tqdm(
                executor.map(get_travel_time_from_row, rows),
                total=len(rows),
                desc="OSRM calls"
            )
        ):
            travel_times[i] = duration

    # ————— Attach results —————
    df_merged['travel_time_sec'] = travel_times
    df_merged['travel_time_min'] = df_merged['travel_time_sec'] / 60

    df_merged.to_csv('../sample_data/test_travel_times.csv',index=False)


In [21]:
df_merged= pd.read_csv('../sample_data/test_travel_times.csv')
df_merged['travel_time_sec'].fillna(1000000,inplace=True)
df_merged['travel_time_min'].fillna(1000000/60,inplace=True)

/var/folders/tf/7k8vj9w17715wv9yqsqn7pm00000gq/T/ipykernel_9422/2585815586.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['travel_time_sec'].fillna(1000000,inplace=True)
/var/folders/tf/7k8vj9w17715wv9yqsqn7pm00000gq/T/ipykernel_9422/2585815586.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

P-median

In [22]:
L=list(grids['cell_id'])
X_exist= list(fire_stations['cell_id'])
# --- toy example ---
E = list(range(len(test_df)))

In [ ]:
a={}

for index, row in test_df.iterrows():
    cluster_label = row['cluster_label']
    if cluster_label in model.model_params:
        w = np.array(model.model_params[cluster_label])
        x = np.array(row[features])
        w_x = np.dot(x, w.T)
        log_t = np.log(row['time_bet'])
        
        # diff = log_t - w_x
        diff=np.clip(log_t - w_x, -500, 500).item()
        likelihood = (diff - np.exp(diff)).sum()
        a[index] = np.exp(likelihood)
    else:
        a[index](np.nan)
    

In [ ]:
import json, urllib.parse
from concurrent.futures import ThreadPoolExecutor, as_completed

from arcgis.gis import GIS

gis = GIS("https://www.arcgis.com", client_id='SpyQX09Wk1E27Afe',client_secret='da00d125df584efd8c96c8f7b36e1a49')

# your two DataFrames
# incident_df must have 'x' & 'y' columns; grid_df same
incident_df = test_df.copy()
grid_df     = grids.copy()

# Base URL
base_url = "https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World/solve"

# travel mode JSON
traffic_travel_mode = {
    "name": "Driving Time with Traffic",
    "impedanceAttributeName": "TravelTime",
    "attributeParameterValues": [],
    "restrictions": []
}

def fetch_travel_time(i, j):
    # build the stops payload
    start_iso = incident_df.at[i, 'AlarmDate'].isoformat()
    stops = {
      "features": [
        {"attributes": {}, "geometry": {
            "x": incident_df.at[i, 'Longitude'],
            "y": incident_df.at[i, 'Latitude']
        }},
        {"attributes": {}, "geometry": {
            "x": grid_df.at[j, 'x'],
            "y": grid_df.at[j, 'y']
        }}
      ]
    }
    params = {
      "stops"           : json.dumps(stops),
      "travelMode"      : json.dumps(traffic_travel_mode),
      "startTime"       : "now",
      "returnDirections": "false",    # only need summary
      "f"               : "json"
    }
    url = base_url + "?" + urllib.parse.urlencode(params)
    resp = gis._con.get(url)

    # pick up the travel time in minutes (field name may vary)
    return resp['routes']['features'][0]['attributes']['Total_TravelTime']

# build all (i,j) pairs
pairs = [(i,j) for i in incident_df.index for j in grid_df.index]

if old_d:
    d={}
    for index,row in df_merged.iterrows():
        i=row['incident_id']
        j=row['fire_station_id']
        d[(i,j)]=row['travel_time_min']

else:
    d = {}
    with ThreadPoolExecutor(max_workers=30) as pool:
        futures = { pool.submit(fetch_travel_time, i, j): (i,j) for (i,j) in pairs }
        for future in as_completed(futures):
            i,j = futures[future]
            try:
                d[(i,j)] = future.result()
            except Exception as e:
                d[(i,j)] = None
                print(f"Error for {(i,j)}:", e)


In [ ]:
# d_loaded=joblib.load('../sample_data/args_pmedian/d.pkl')
# X_exist_loaded=joblib.load('../sample_data/args_pmedian/X_exist.pkl')
# L_loaded=joblib.load('../sample_data/args_pmedian/L.pkl')
# a_loaded=joblib.load('../sample_data/args_pmedian/a.pkl')
# E_loaded=joblib.load('../sample_data/args_pmedian/E.pkl')


In [ ]:
#sort by i,j


In [ ]:
# 1. Sort your dict items by (i, j)
sorted_items = sorted(d.items(), key=lambda x: (x[0][0], x[0][1]))

# 2. Extract the sorted list of unique i’s
unique_i = sorted({i for (i, _), _ in sorted_items})

# 3. Take the first half of those i’s
half_count = len(unique_i) // 4
first_i_set = set(unique_i[:half_count])

# 4. Filter to only those entries whose i is in that first half
first_half_by_i = {
    (i, j): val
    for (i, j), val in sorted_items
    if i in first_i_set
}

a_sub = {k: a[k] for k in a if k < half_count}
# model, X, Y, b = add_p_via_mip_multi(E[:half_count], L, a_sub, first_half_by_i, X_exist, p_add=0, alpha=0)


In [ ]:

# joblib.dump(first_half_by_i, '../sample_data/args_pmedian/first_half_by_i.pkl')
# joblib.dump(a_sub, '../sample_data/args_pmedian/a_sub.pkl')

In [ ]:
#subdict of d
a_sub = {k: a[k] for k in a if k < half_count}
# a_sub=a

In [ ]:
# import gurobipy as gp
# from gurobipy import GRB
# model2, X2, Y2, b = add_p_via_mip_multi(E, L, a, d, X_exist, p_add=1, alpha=0)

In [ ]:
# model, X, Y, b = add_p_via_mip_multi(E, L, a, d, X_exist, p_add=0, alpha=0)
model, X, Y, b = add_p_via_mip_multi(E[:half_count], L, a_sub, first_half_by_i, X_exist, p_add=0, alpha=0)

In [ ]:
import os
import pandas as pd
import json
import pickle

def save_p_median_solution(
    X, Y, 
    E, L,
    output_dir=".",
    prefix="solution",
    to_csv=True,
    to_json=True,
    to_pickle=True
):
    """
    Persist a p-median MIP solution (X, Y) in various formats.

    Parameters
    ----------
    X : dict-like of Gurobi vars
        X[j].X gives the integer count at location j.
    Y : dict-like of Gurobi vars
        Y[i,j].X gives the binary assignment of demand i to location j.
    E : iterable
        Demand indices.
    L : iterable
        Candidate‐location indices.
    output_dir : str, default="."
        Directory to write files into (will be created if necessary).
    prefix : str, default="solution"
        Filename prefix (e.g. "solution" → "solution_X.csv", etc.).
    to_csv : bool, default=True
        Whether to write CSV files (`{prefix}_X.csv`, `{prefix}_Y.csv`).
    to_json : bool, default=True
        Whether to write `{prefix}.json`.
    to_pickle : bool, default=True
        Whether to write `{prefix}.pkl`.

    Returns
    -------
    dict
        The Python objects that were saved: 
        {
          "X_vals": {j: int},
          "Y_assign": {i: j}
        }
    """
    # ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # extract solution values
    X_vals = { j: int(X[j].X) for j in L }
    Y_assign = {
        i: j
        for i in E
        for j in L
        if Y[i,j].X > 0.5
    }

    # CSV output
    if to_csv:
        df_X = pd.DataFrame.from_dict(
            X_vals, orient="index", columns=["count"]
        )
        df_X.index.name = "location"
        df_X.to_csv(os.path.join(output_dir, f"{prefix}_X.csv"))

        df_Y = pd.DataFrame([
            {"demand": i, "location": j}
            for i, j in Y_assign.items()
        ])
        df_Y.to_csv(os.path.join(output_dir, f"{prefix}_Y.csv"), index=False)

    # JSON output
    if to_json:
        with open(os.path.join(output_dir, f"{prefix}.json"), "w") as f:
            json.dump({"X": X_vals, "Y": Y_assign}, f, indent=2)

    # Pickle output
    if to_pickle:
        with open(os.path.join(output_dir, f"{prefix}.pkl"), "wb") as f:
            pickle.dump((X_vals, Y_assign), f)

    return {"X_vals": X_vals, "Y_assign": Y_assign}

# model2.write("model2.lp")
model.write("model.lp")
# save_p_median_solution(X2, Y2, E, L, output_dir="../p_median", prefix="solution2")
save_p_median_solution(X, Y, E, L, output_dir="../p_median", prefix="solution")